## CSV Preprocessing

The modified csvs from this notebook are in data/processed.

In [1]:
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os
from PIL import Image
import importlib
import pandas as pd

In [29]:
from csv_Extractor_Rohan import Extractor 

In [30]:
path = "D:/Rohan/04_DL/CBIS-DDSM/csv"
base_path = "D:/Rohan/04_DL/CBIS-DDSM/"

In [31]:
test = Extractor(path)

In [32]:
test.get_file_names()

['calc_case_description_test_set.csv',
 'calc_case_description_train_set.csv',
 'dicom_info.csv',
 'mass_case_description_test_set.csv',
 'mass_case_description_train_set.csv',
 'meta.csv']

In [33]:
test.load_df()

In [34]:
test.rename("mass_case_description_train_set.csv","mass_train")
test.rename("calc_case_description_train_set.csv","calc_train")

In [35]:
test.dataframes.keys()

dict_keys(['calc_case_description_test_set.csv', 'dicom_info.csv', 'mass_case_description_test_set.csv', 'meta.csv', 'mass_train', 'calc_train'])

In [9]:
test.dataframes["mass_train"]["ROI mask file path"].iloc[2]

'Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.414182170112396175925115449620455230167/1.3.6.1.4.1.9590.100.1.2.429120414011832984817094399141838850375/000001.dcm\n'

In [10]:
test.preprocess_steps(filename=["calc_train","mass_train"],image_path=base_path)

In [11]:
test.modified_dataframe.keys()

dict_keys(['calc_train', 'mass_train'])

In [12]:
calc_train = test.modified_dataframe["calc_train"]
mass_train = test.modified_dataframe["mass_train"]

In [13]:
mass_train.head()


,patient_id,assessment,pathology,image file path,ROI mask file path,ROI image,full image
0,P_00001,4,MALIGNANT,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
1,P_00001,4,MALIGNANT,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
2,P_00004,4,BENIGN,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
3,P_00004,4,BENIGN,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
4,P_00004,4,BENIGN,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...


In [21]:
print(len(mass_train))

1318


In [17]:
print(mass_train.isnull().sum())

patient_id            0
assessment            0
pathology             0
image file path       0
ROI mask file path    0
ROI image             0
full image            0
dtype: int64


## Small Model Test

In [5]:
mass_train = pd.read_csv("D:/Rohan/04_DL/mod_csv/mass_train.csv")
calc_train = pd.read_csv("D:/Rohan/04_DL/mod_csv/calc_train.csv")

In [ ]:
#importlib.reload(model_Library_Rohan)

<module 'model_Library_Rohan' from 'c:\\Users\\r.sujeesh\\OneDrive - enmodes GmbH\\PyAnsysnotebooks\\DL\\model_Library_Rohan.py'>

In [9]:
import model_Library_Rohan

In [21]:
model = model_Library_Rohan.small_unet_test()

In [22]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
#create sample tensor for unet model feeding
n = len(mass_train)
image = np.ones((n,256,256,1))
mask = np.ones((n,256,256,1))


In [18]:
#Normalize and add grayscale channel, resize to unet model input
for i, path in enumerate (mass_train["full image"]):
    im = Image.open(path)
    im=im.resize((256,256))
    im = np.asarray(im) / 255.0  
    im = np.expand_dims(im, axis=-1)  
    image[i]=im

In [19]:
for j, path in enumerate (mass_train["ROI image"]):
    im = Image.open(path)
    im=im.resize((256,256))
    im = np.asarray(im) / 255.0  # normalize
    im = np.expand_dims(im, axis=-1)  # add channel
    mask[j]=im

In [23]:
train_set = (image,mask)

In [27]:
smaple = image[0]
masknew = mask[0]

In [28]:
smaple_im = (smaple.squeeze()*255).astype(np.uint8)
mask_im = (masknew.squeeze()*255).astype(np.uint8)

In [29]:
smaple_imm = Image.fromarray(smaple_im)
mask_imm =  Image.fromarray(mask_im)

In [30]:
smaple_imm.show()
mask_imm.show()

In [26]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model.fit(train_set[0],train_set[1],batch_size=16,epochs=50)

Epoch 1/50
83/83 [==============================] - 16s 188ms/step - loss: 0.0186 - accuracy: 0.9939 - dice_coef: 0.1418
Epoch 2/50
83/83 [==============================] - 16s 189ms/step - loss: 0.0175 - accuracy: 0.9940 - dice_coef: 0.1842
Epoch 3/50
83/83 [==============================] - 16s 190ms/step - loss: 0.0171 - accuracy: 0.9940 - dice_coef: 0.1933
Epoch 4/50
83/83 [==============================] - 16s 190ms/step - loss: 0.0165 - accuracy: 0.9942 - dice_coef: 0.2212
Epoch 5/50
83/83 [==============================] - 16s 192ms/step - loss: 0.0163 - accuracy: 0.9943 - dice_coef: 0.2339
Epoch 6/50
83/83 [==============================] - 16s 193ms/step - loss: 0.0155 - accuracy: 0.9944 - dice_coef: 0.2561
Epoch 7/50
83/83 [==============================] - 16s 193ms/step - loss: 0.0153 - accuracy: 0.9944 - dice_coef: 0.2650
Epoch 8/50
83/83 [==============================] - 16s 194ms/step - loss: 0.0151 - accuracy: 0.9944 - dice_coef: 0.2784
Epoch 9/50
83/83 [==============


##### Remarks in their order of occurence:

1. Image size (512,512) along with a unet of deepest layer with kernel size 1024 is causing OOM error for my GPU specs, if i were to run a batch size of 8. It runs at most with batch size 4.

2. Image size (256,256) with deepest layer of size 512 reduces memory load enough to run batch size 8. Maybe deeper layers would be possible with this image size.

3. The immediate result of high accuracy was suspicious. I realized that the segmentation problem we are posing is highly imbalanced. The binary masks are imabalanced, weighted towards the background, with smaller regions of interest. This makes the model's job easy, in the sense that it can with high accuracy just predict background, which is not valid.

4. This needs to be addressed, with different possible approaches:
    - Add different loss metric functions, that adjust for this imbalance. One such i have already tried to implement (please refer to code in model Library) called dice coefficient loss. This metric shows that our model doesnt actually have high accuracy, rather really poor (see the numbers in the epochs above, a good model should be closer to 1!).

    - The dataset needs to be evaluated in patches, so that ROIs spatially cover more of each patch.

    - a sliding window approach is used.

#### Possible tasks to complete:

1. Create a good dataloader function to load data in patches,etc.

2. define appropriate loss and model functions.

...

In [36]:
test.dataframes.keys()

dict_keys(['calc_case_description_test_set.csv', 'dicom_info.csv', 'mass_case_description_test_set.csv', 'meta.csv', 'mass_train', 'calc_train'])

In [37]:
test.preprocess_steps(filename=["mass_case_description_test_set.csv"],image_path=base_path)

In [40]:
mass_test = test.modified_dataframe["mass_case_description_test_set.csv"]

In [41]:
mass_test.head()

,patient_id,assessment,pathology,image file path,ROI mask file path,ROI image,full image
0,P_00016,5,MALIGNANT,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
1,P_00016,5,MALIGNANT,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
2,P_00017,4,MALIGNANT,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
3,P_00017,4,MALIGNANT,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...
4,P_00032,0,BENIGN,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...,D:/Rohan/04_DL/CBIS-DDSM/jpeg/1.3.6.1.4.1.9590...


In [48]:
sample_image = Image.open(mass_test["full image"].iloc[0])
sample_image = sample_image.resize((256,256))
sample_image = np.asarray(sample_image)
sample_image = np.expand_dims(sample_image,axis=-1)

real_mask = Image.open(mass_test["ROI image"].iloc[0])

In [49]:
pred_mask = model.predict(sample_image)

ValueError: in user code:

    File "c:\Users\r.sujeesh\AppData\Local\miniconda3\envs\tf-2.10-ds\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\r.sujeesh\AppData\Local\miniconda3\envs\tf-2.10-ds\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\r.sujeesh\AppData\Local\miniconda3\envs\tf-2.10-ds\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\r.sujeesh\AppData\Local\miniconda3\envs\tf-2.10-ds\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\r.sujeesh\AppData\Local\miniconda3\envs\tf-2.10-ds\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro